In [25]:
import pandas as pd
import numpy as np
import ast
import warnings
import csv

pd.set_option("display.max_columns", None) # MUESTRA TODAS LAS COLUNAS DE UN PANDAS DATAFRAME
warnings.filterwarnings('ignore')

API_KEY = 'GAN1vC5xqu0S0lE04dNYZtbBEFtqWTayl0ulFadV'
URL_API = 'http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/'

In [18]:
# Acceso a Internet Fijo por rangos de velocidad de bajada y provincia
GUI = 'ACCES-A-INTER-FIJO-23248'
url = URL_API + GUI + '/data.csv/?auth_key=' + API_KEY
file = 'Datasets/Acceso_Internet_Fijo_por_rangos_velocidad_bajada_y_provincia.csv'
try:
    df_velocidad = pd.read_csv(url, sep=",", encoding='utf_8', thousands='.', decimal=',')
except:
    df_velocidad = pd.read_csv(file, sep=",", encoding='utf_8', thousands='.', decimal=',')

# Penetración del Internet fijo por provincia (accesos por cada 100 habitantes)
GUI = 'PENET-DEL-INTER-FIJO-POR'
url = URL_API + GUI + '/data.csv/?auth_key=' + API_KEY
file = 'Datasets/Penetración_por_hogares_nacional_Internet fijo.csv'
try:
    df_acceso_habitante = pd.read_csv(url, sep=",", encoding='utf_8', thousands='.', decimal=',')
except:
    df_acceso_habitante = pd.read_csv(file, sep=",", encoding='utf_8', thousands='.', decimal=',')

# Penetración del Internet fijo por provincia (accesos por cada 100 hogares)
GUI = 'PENET-DEL-INTER-FIJO-51614'
url = URL_API + GUI + '/data.csv/?auth_key=' + API_KEY
file = 'Datasets/Penetración_de _Internet fijo _(accesos _cada_100 hogares).csv'
try:
    df_acceso_hogar = pd.read_csv(url, sep=",", encoding='utf_8', thousands='.', decimal=',')
except:
    df_acceso_hogar = pd.read_csv(file, sep=",", encoding='utf_8', thousands='.', decimal=',')

# Accesos a Internet fijo por velocidad bajada y provincia
GUI = 'ACCES-A-INTER-FIJO-97624'
url = URL_API + GUI + '/data.csv/?auth_key=' + API_KEY
file = 'Datasets/Velocidad_media_bajada_Internet_fijo_por_provincia.csv'
try:
    df_velocidad_media = pd.read_csv(url, sep=",", encoding='utf_8', thousands='.', decimal=',')
except:
    df_velocidad_media = pd.read_csv(file, sep=",", encoding='utf_8', thousands='.', decimal=',')

In [17]:
df_velocidad['OTROS'].isnull().sum()

6

In [19]:
# ETL en dataframe df_velocidad
df_velocidad['OTROS'].fillna(0, inplace=True)
velocidad = df_velocidad['HASTA 512 kbps'] + df_velocidad['+ 512 Kbps - 1 Mbps'] + df_velocidad['+ 1 Mbps - 6 Mbps'] + df_velocidad['+ 6 Mbps - 10 Mbps']
df_velocidad.insert(loc=3, column='+ 0 Mbps - 10 Mbps', value=velocidad)
df_velocidad.drop(['HASTA 512 kbps', '+ 512 Kbps - 1 Mbps', '+ 1 Mbps - 6 Mbps', '+ 6 Mbps - 10 Mbps'], axis=1, inplace=True)

# Eliminando filas que no existen en algunos dataframes
df_velocidad.drop(df_velocidad[df_velocidad['Año'] >= 2022].index, inplace=True)
df_acceso_habitante.drop(df_acceso_habitante[df_acceso_habitante['Año'] >= 2022].index, inplace=True)
df_acceso_hogar.drop(df_acceso_hogar[df_acceso_hogar['Año'] >= 2022].index, inplace=True)
df_velocidad_media.drop(df_velocidad_media[df_velocidad_media['Año'] >= 2022].index, inplace=True)

In [24]:
# Creando tabla hechos, join de los dataframes
df_internet = df_velocidad.merge(df_acceso_habitante,on=['Año', 'Trimestre', 'Provincia']).merge(df_acceso_hogar,on=['Año', 'Trimestre', 'Provincia']).merge(df_velocidad_media,on=['Año', 'Trimestre', 'Provincia'])

# Creando dimension Provincia
df_provincia = df_velocidad.groupby(['Provincia'])['Total'].count().reset_index()
df_provincia = df_provincia[['Provincia']]
df_provincia['ProvinciaPais'] = 'Province ' + df_provincia['Provincia'] + ', Argentina'

# Creando dimension Calendario
df_calendario = df_velocidad.groupby(['Año', 'Trimestre'])['Total'].count().reset_index()
df_calendario = df_calendario[['Año', 'Trimestre']].sort_values(by='Año', ascending=False)

In [32]:
df_internet.to_csv('Datasets/internet.csv', index=False, sep=';')
df_provincia.to_csv('Datasets/dim_provincia.csv', index=False, sep=';')
df_calendario.to_csv('Datasets/dim_calendario.csv', index=False, sep=';')

# SQL Creacion de tablas en MySql y load de data

```
-- ---------------------------------------
-- CREANDO TABLAS
-- ---------------------------------------
DROP TABLE IF EXISTS `dim_calendario`;
CREATE TABLE IF NOT EXISTS `dim_calendario` (
  	`Año` 			INTEGER,
  	`Trimestre` 	INTEGER
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_spanish_ci;	

DROP TABLE IF EXISTS `dim_provincia`;
CREATE TABLE IF NOT EXISTS `dim_provincia` (
  `Provincia`			VARCHAR(50),
  `ProvinciaPais`		VARCHAR(100)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_spanish_ci;

DROP TABLE IF EXISTS `internet`;
CREATE TABLE IF NOT EXISTS `internet` (
  	`Año` 						INTEGER,
  	`Trimestre` 				INTEGER,
  	`Provincia`					VARCHAR(100),
	`+ 0 Mbps - 10 Mbps`		INTEGER,
	`+ 10 Mbps - 20 Mbps`		INTEGER,
	`+ 20 Mbps - 30 Mbps`		INTEGER,
	`+ 30 Mbps`					INTEGER,
	`Otros`						DECIMAL(10,2),
	`Total`						INTEGER,
	`Accesos por cada 100 hab`		INTEGER,
	`Accesos por cada 100 hogares`	DECIMAL(10,2),
	`Mbps (Media de bajada)`		INTEGER
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_spanish_ci;

ALTER TABLE dim_calendario ADD PRIMARY KEY (`Año`, `Trimestre`);
ALTER TABLE dim_provincia ADD PRIMARY KEY (`Provincia`);
ALTER TABLE internet ADD PRIMARY KEY (`Año`, `Trimestre`, `Provincia`);



-- ---------------------------------------
-- LOAD CSV A TABLAS
-- ---------------------------------------

SELECT @@global.secure_file_priv; -- Para ver la ruta de origen donde poner los csv.

LOAD DATA INFILE 'csv\\da_internet\\dim_calendario.csv'
INTO TABLE dim_calendario  
CHARACTER SET utf8mb4 -- Si no colocamos esta línea, no reconoce la codificación adecuada ANSI
FIELDS TERMINATED BY ';' ENCLOSED BY '' ESCAPED BY '' 
LINES TERMINATED BY '\n' IGNORE 1 LINES;

SELECT * FROM dim_calendario;

LOAD DATA INFILE 'csv\\da_internet\\dim_provincia.csv'
INTO TABLE dim_provincia  
CHARACTER SET utf8mb4 -- Si no colocamos esta línea, no reconoce la codificación adecuada ANSI
FIELDS TERMINATED BY ';' ENCLOSED BY '' ESCAPED BY '' 
LINES TERMINATED BY '\n' IGNORE 1 LINES;

SELECT * FROM dim_provincia;

LOAD DATA INFILE 'csv\\da_internet\\internet.csv'
INTO TABLE internet  
CHARACTER SET utf8mb4 -- Si no colocamos esta línea, no reconoce la codificación adecuada ANSI
FIELDS TERMINATED BY ';' ENCLOSED BY '"' ESCAPED BY '' 
LINES TERMINATED BY '\n' IGNORE 1 LINES;

SELECT * FROM internet;

```